In [4]:
# installing dependencies
import mne
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np

In [5]:
def baseline_chunking(event_idx, data):
    baseline_data = []
    for idx in event_idx:
        channel_data = []
        for channel in data:
            channel_data.append(channel[idx-500:idx])
        baseline_data.append(channel_data)
    return baseline_data

In [6]:
def event_chunking(event_idx,data,offset=250):
    event_data = []
    for idx in event_idx:
        channel_data = []
        for channel in data:
            channel_data.append(channel[idx+offset:idx+offset+500])
        event_data.append(channel_data)
    return event_data

In [7]:
'''
Obtaining raw data from the BCICIV data
Data consists of mainly right and left hand motions
but also constitutes moments of no motion (baseline)
'''
raw = mne.io.read_raw_gdf('BCICIV_2a_gdf\A01T.gdf', preload=True)

Extracting EDF parameters from c:\Users\aliz0\OneDrive\Documents\GitHub\EEGTetris\ml\BCICIV_2a_gdf\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...


c:\Users\aliz0\anaconda3\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


In [8]:
# Obtaining all channel names 
channel_names = raw.info['ch_names']
#print(channel_names)

#Obtaining a subset of those channels
channels_to_keep = ['EEG-1', 'EEG-3', 'EEG-C3', 'EEG-C4', 'EEG-14', 'EEG-Pz', 'EEG-15']     # F3, F4, C3, C4, Pz, P3, P4

raw_subset = raw.copy().pick(channels_to_keep)

In [9]:
'''
order_to_class_id = {
    '1' : 'Idling Eyes Open',
    '2' : 'Idling Eyes Closed',
    '3' : 'Trial Start',
    '4' : 'Cue Left',
    '5' : 'Cue Right',
    '6' : 'Cue Foot',
    '7' : 'Cue Tongue', 
    '8' : 'Cue Unknown',
    '9' : 'Rejected Trial',
    '10': 'Eye Movements',
    '11': 'Next Run'
}

og_event_id = {
    '276'  : 'Idling Eyes Open',
    '277'  : 'Idling Eyes Closed',
    '768'  : 'Trial Start',
    '769'  : 'Cue Left',
    '770'  : 'Cue Right',
    '771'  : 'Cue Foot',
    '772'  : 'Cue Tongue', 
    '783'  : 'Cue Unknown',
    '1023' : 'Rejected Trial',
    '1072' : 'Eye Movements',
    '32766': 'Next Run'
}
'''


ordered_event_id = {
    '276'  : 1,
    '277'  : 2,
    '768'  : 3,
    '769'  : 4,
    '770'  : 5,
    '771'  : 6,
    '772'  : 7, 
    #'783'  : 8,
    '1023' : 9,
    '1072' : 10,
    '32766': 11
}

events, event_id = mne.events_from_annotations(raw_subset, event_id = ordered_event_id)
# events.shape, events[:10]

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


In [10]:
events_df = pd.DataFrame(events, columns = ["sample_index", "prev_signal", "event_code"])
event_idx = events_df[events_df['event_code'] == 4]['sample_index']

In [13]:
# read data for chunking
data = raw_subset.get_data()
type(data), data.shape

# chunking
baseline_data = baseline_chunking(event_idx,data)
event_data = event_chunking(event_idx,data)

np.array(baseline_data).shape, np.array(event_data).shape

((72, 7, 500), (72, 7, 500))